

# Scotiabank Quarterly Reports: Instant Analysis and Translation Powered by Custom Made Chatbot

Source: https://www.scotiabank.com/ca/en/about/investors-shareholders/financial-result.html

## Team Alpha

* Raemil Corniel
* Uma Maheshwari
* Sonal Parmar
* Jharana Adhikari
* Aparna Suresh1


Content
0.  Install libraries needed to run the notebook
1.  Load Libraries
2.  Class and functions
3.  Load data
4.  Analysis data (before cleaning)
5.  Clean data
6.  





### 0. Install libraries needed to run the notebook

In [12]:
pip install tabula-py pypdf PdfReader pdfminer pdfminer.six jpype1


### 1. Load Libraries

In [19]:
import pandas as pd
import re
import math

from pypdf import PdfReader
from pdfminer.converter import TextConverter
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.layout import LAParams
from pdfminer.high_level import extract_text

import io
import tabula
import jpype

from bs4 import BeautifulSoup

In [280]:
#show all columns in a df
pd.set_option('display.max_columns', None)
#show full row info
pd.set_option('display.max_colwidth', None)

In [14]:
print("JPype version:", jpype.__version__)


JPype version: 1.5.0


### 2. Classes and Function
#### This is to avoid code repetition and for code clarity and reusability  

In [48]:
#load file, given a pdf file path
def load_file(pdf):
    #load the file
    document = PdfReader(pdf , 'rb')

    counter=0
    textList = []

    # only read first 11 pages the last 3 pages are not relevant (Forward-looking statements, Shareholders Information)
    while counter < (len(document.pages) -3):
        pageObj = document.pages[counter]

        textList.append(pageObj.extract_text())
        counter += 1

    #return a list with the pages of the document, this list does not contain tables only textual info
    return textList


In [49]:
#clean text from html tags, \n, extra spaces, urls, double spaces , lower case
def clean_text(fileText):

    #clean file text
    textCleannedList = []
    for textToProccess in fileText:
        #lower case
        textToProccess = textToProccess.lower()

        #remove all tags from html
        textToProccess= BeautifulSoup(textToProccess, 'html.parser').get_text()

        #remove newlines
        textToProccess= textToProccess.replace('\n',' ')

        #remove spaces
        textToProccess = textToProccess.strip()

        #remove urls
        patterns = re.compile(r'https?://\S+|www\.\S+')
        textToProccess = patterns.sub(' ', textToProccess)

        #remove double space
        textToProccess =re.sub("\s\s+" , " ", textToProccess)

        textCleannedList.append(textToProccess)

    return textCleannedList

In [72]:
#get text from specific part of the document
#given a list of pages, start word, end word , and a tag for identifying type of information extracted
def extract_text_between_words_across_pages(text_pages, start_word, end_word, tag):
    extracted_texts = []
    #join all pages
    combined_text = "\n".join(text_pages)
    pattern = re.compile(re.escape(start_word) + '(.*?)' + re.escape(end_word), re.S)

    for match in pattern.finditer(combined_text):
        start_page, end_page = None, None
        start_index, end_index = match.start(), match.end()

        cumulative_length = 0
        for i, page in enumerate(text_pages):
            page_length = len(page)
            if start_page is None and start_index < cumulative_length + page_length:
                start_page = i + 1
            if end_page is None and end_index <= cumulative_length + page_length:
                end_page = i + 1
            cumulative_length += page_length + len("\n")
            if start_page is not None and end_page is not None:
                break

        extracted_texts.append({
            'start_page': start_page,
            'end_page': end_page,
            'start_word': start_word,
            'end_word': end_word,
            'tag':tag,
            'extracted_text': start_word +' '+ match.group(1).strip()
        })
    return extracted_texts

In [73]:
#given a list of pages, and  list of set of words (start, end) return a list of findings
def process_files_in_folder(text_pages, start_end_pairs, tag):
    all_extracted_texts = []

    for start_word, end_word in start_end_pairs:
        extracted_texts = extract_text_between_words_across_pages(text_pages, start_word, end_word,tag)
        all_extracted_texts.extend(extracted_texts)

    return all_extracted_texts


### 3. Load data



> In this step the data will be load twice, once using the library pypdf for gather all the text of the report, and using tabula, this library will help us load all the tables of the report in a more propper format.



In [132]:
# Replace 'your_pdf_file.pdf' with the path to your PDF file
pdf_path = "Q224.pdf"

#get file text , provide pdf path
file_text = load_file(pdf_path)




# Extract all tables from the PDF
tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)



#indicate the quater and year
quater= 'second'
year = '2024'

### 4. Analysis of the data (before cleaning)

In [130]:
#len of the document
print('Total pages', len(file_text))


#words per page
for index , page in enumerate(file_text):
    words_per_page = len(page.split())
    print(f'Total words page #{index} :{ words_per_page}')

#amount of words in the document
total_words = sum(len(page.split()) for page in file_text)
print('\nTotal Words:', total_words)



print(f'Total tables available: {len(tables)}')


Total pages 11
Total words page #0 :663
Total words page #1 :726
Total words page #2 :959
Total words page #3 :1042
Total words page #4 :252
Total words page #5 :379
Total words page #6 :574
Total words page #7 :644
Total words page #8 :634
Total words page #9 :739
Total words page #10 :761

Total Words: 7373
Total tables available: 6




> The financial Quarterly Release report of Scotiabank has ~14 pages but, only the frist 11 has been taken for analysis, since the last pages do not provide useful information



*   Total pages 11
*   Total Words: 7373
*   Tables: 6

> Actually the report has 10 tables, which means that 4 tables are not read correctly, in the next steps this will be fixed
















### 5. Clean the data




> In this step the data obtained from the pdf will be cleaned and adapted for proper use, with that said the next steps will be applied:

* Remove HTML tags
* Lower case text
* remove \n
* remove url
* remove extra spaces



> Similar steps will be applied again for extracting and cleaning the tables





In [53]:
#show lenght of document
print('Document total pages:',len(file_text))
#by showing second page we expect to see a tables, in text format
print ('Second page: ')
file_text[1]


Document total pages: 11
Second page: 


' \n                     \n            \n                \n            \n           \n                \n          \n                  \n                \n                 \n          \n                      \n                \n         \n           \n              \n          \n         \n \n                                 \n          \n \n \n     \n \n                    \n                   \n                         \n                          \n           \n \n    \n   \n      \n                        \n                \n      \n                         \n             \n        \n                       \n                       \n   \n      \n                      \n                         \n                \n      \n                       \n                       \n                        \n      \n        \n                       \n                        \n                       \n Financial  Highlights  \nReported Results For the three months ended For the six months ended \n

In [54]:
#return a list with text cleanned remove   html tags, \n, extra spaces, urls, double spaces , lower case
text_cleanned = clean_text(file_text)

In [55]:
text_cleanned[1] #show frist page after cleanned
#look that the table of financial highlights is visible, we will remove this table
#since it will be loaded later in a df with the right format

'financial highlights reported results for the three months ended for the six months ended (unaudited) ($ millions) april 30 2024(1) january 31 2024(1) april 30 2023(1) april 30 2024(1) april 30 2023(1) operating results net interest income $ 4,694 $ 4,773 $ 4,460 $ 9,467 $ 9,023 non -interest income 3,653 3,660 3,453 7,313 6,852 total revenue $ 8,347 $ 8,433 $ 7,913 $ 16,780 $ 15,875 provision for credit losses 1,007 962 709 1,969 1,347 non -interest expenses 4,711 4,739 4,574 9,450 9,035 income tax expense 537 533 484 1,070 1,589 net income $ 2,092 $ 2,199 $ 2,146 $ 4,291 $ 3,904 net income attributable to non-controlling interests in subsidiaries 26 25 24 51 61 net income attributable to equity holders of the bank $ 2,066 $ 2,174 $ 2,122 $ 4,240 $ 3,843 preferred shareholders and other equity instrument holders 123 108 104 231 205 common shareholders $ 1,943 $ 2,066 $ 2,018 $ 4,009 $ 3,638 earnings per common share (in dollars) basic $ 1.59 $ 1.70 $ 1.69 $ 3.29 $ 3.05 diluted $ 1.57



> For better understanding of the document, it will be separated into chunks of information, which will give us information classified and organized according to the report, this way we can easily search for key topics like, global banking and markets.



> A list of sets will be created indicating the start and end of the chunks desire







In [76]:
#tables we will removed from text since it will imported using tabula

#create a list of set with keyword that indicates the begining and end of a part of
#information of in the document.


summary_set = [
    (quater + ' quarter ' + year+' earnings release' , quater + ' quarter ' + year+' highlights on a reported basis'),
    (quater + ' quarter ' + year+' highlights on a reported basis' , quater + ' quarter ' + year+' highlights on an adjusted basis'),
    (quater + ' quarter ' + year+' highlights on an adjusted basis' , 'toronto,'),
    ('toronto,' , 'financial highlights'),
]


financial_ighlights  = [
    ('reported results ', 'business segment review'),
]

business_segment_review = [
    ('business segment review', 'credit risk'),
    ('canadian banking', 'international banking'),
    ('international banking', 'financial performance on a constant dollar basis'),
    ('financial performance on a constant dollar basis','global wealth management'),
    ('global wealth management', 'global banking and markets'),
    ('global banking and markets', 'other'),
    ('other', 'credit risk'),
]

credit_risk = [
    ('credit risk','capital ratios'),
    ('provision for credit losses', 'allowance for credit losses'),
    ('allowance for credit losses', 'impaired loans'),
    ('impaired loans','capital ratios'),
]

capital_ratios = [
    ('capital ratios','non-gaap measures')
]

non_gaap_measures = [
    ('non-gaap measures','reconciliation of reported and adjusted results and diluted earnings per share'),
    ('non-gaap measures','adjusted results and diluted earnings per share'),
    ('adjusted results and diluted earnings per share','reconciliation of reported and adjusted results and diluted earnings per share'),


]

reconciliation= [
    ('reconciliation of reported and adjusted results and diluted earnings per share','reconciliation of reported and adjusted results by business line'),
    ('reconciliation of reported and adjusted results by business line','reconciliation of International Banking’sreported, adjusted and constant dollar results'),
    ('reconciliation of International Banking’sreported, adjusted and constant dollar results', 'return on equity'),
]

return_on_equity = [
        ('return on equity','return on equity by operating segment'),
        ('return on equity by operating segment','forward-looking statements'),

]



In [112]:
#information used for report: list of dictionaries

summary_part =  process_files_in_folder(text_cleanned, summary_set, 'summary')
financial_ighlights_part =  process_files_in_folder(text_cleanned, financial_ighlights, 'financial_ighlights')
business_segment_review_part =  process_files_in_folder(text_cleanned, business_segment_review, 'business_segment_review')
credit_risk_part =  process_files_in_folder(text_cleanned, credit_risk, 'credit_risk')
capital_ratios_part =  process_files_in_folder(text_cleanned, capital_ratios, 'capital_ratios')
non_gaap_measures_part =  process_files_in_folder(text_cleanned, non_gaap_measures, 'non-gaap-measures')
reconciliation_part =  process_files_in_folder(text_cleanned, reconciliation, 'reconciliation')
return_on_equity_part =  process_files_in_folder(text_cleanned, return_on_equity, 'return_on_equity')





> For ease of use, the chunks have been converted to a dataframe.




In [113]:

# Convert each list of dictionaries to a DataFrame
summary_df = pd.DataFrame(summary_part)
financial_highlights_df = pd.DataFrame(financial_ighlights_part)
business_segment_review_df = pd.DataFrame(business_segment_review_part)
credit_risk_df = pd.DataFrame(credit_risk_part)
capital_ratios_df = pd.DataFrame(capital_ratios_part)
non_gaap_measures_df = pd.DataFrame(non_gaap_measures_part)
reconciliation_df = pd.DataFrame(reconciliation_part)
return_on_equity_df = pd.DataFrame(return_on_equity_part)

# Combine all DataFrames into one
finnacial_release_df = pd.concat([
    summary_df,
    financial_highlights_df,
    business_segment_review_df,
    credit_risk_df,
    capital_ratios_df,
    non_gaap_measures_df,
    reconciliation_df,
    return_on_equity_df
], ignore_index=True)


print('Shape',finnacial_release_df.shape)

Dimenssions (51, 6)


In [116]:
finnacial_release_df.head() #get first 5 rows

,start_page,end_page,start_word,end_word,tag,extracted_text
0,1,1,second quarter 2024 earnings release,second quarter 2024 highlights on a reported b...,summary,second quarter 2024 earnings release scotiaban...
1,1,1,second quarter 2024 highlights on a reported b...,second quarter 2024 highlights on an adjusted ...,summary,second quarter 2024 highlights on a reported b...
2,1,1,second quarter 2024 highlights on an adjusted ...,"toronto,",summary,second quarter 2024 highlights on an adjusted ...
3,1,2,"toronto,",financial highlights,summary,"toronto, may 28, 2024 —the bank of nova scotia..."
4,2,2,reported results,business segment review,financial_ighlights,reported results for the three months ended f...


In [118]:
finnacial_release_df.tail(10) # get last 10 rows


,start_page,end_page,start_word,end_word,tag,extracted_text
41,1,4,other,credit risk,business_segment_review,other wise noted. our complete second quarter ...
42,4,5,credit risk,capital ratios,credit_risk,credit risk provision for credit losses q2 202...
43,1,4,provision for credit losses,allowance for credit losses,credit_risk,provision for credit losses increased compared...
44,4,4,allowance for credit losses,impaired loans,credit_risk,allowance for credit losses the total allowanc...
45,4,5,impaired loans,capital ratios,credit_risk,"impaired loans was $975 million, compared to $..."
46,1,6,capital ratios,non-gaap measures,capital_ratios,capital ratios are based on revised basel iii ...
47,6,7,non-gaap measures,reconciliation of reported and adjusted result...,non-gaap-measures,non-gaap measures and ratios are useful as the...
48,6,7,non-gaap measures,adjusted results and diluted earnings per share,non-gaap-measures,non-gaap measures and ratios are useful as the...
49,7,8,reconciliation of reported and adjusted result...,reconciliation of reported and adjusted result...,reconciliation,reconciliation of reported and adjusted result...
50,1,11,return on equity,return on equity by operating segment,return_on_equity,"return on equity (2) of 11.2%, compared to 12...."


In [119]:
#rows with no meaning
finnacial_release_df[finnacial_release_df['extracted_text'].apply(len) <100]

,start_page,end_page,start_word,end_word,tag,extracted_text
11,6,6,canadian banking,international banking,business_segment_review,"canadian banking ,"
12,8,8,canadian banking,international banking,business_segment_review,canadian banking (2)
13,8,8,canadian banking,international banking,business_segment_review,canadian banking (2)
14,9,9,canadian banking,international banking,business_segment_review,canadian banking (2)
15,9,9,canadian banking,international banking,business_segment_review,canadian banking (2)
16,10,10,canadian banking,international banking,business_segment_review,canadian banking (2)
17,11,11,canadian banking,international banking,business_segment_review,canadian banking (1)
18,11,11,canadian banking,international banking,business_segment_review,canadian banking (1)
25,8,8,global wealth management,global banking and markets,business_segment_review,global wealth management
26,9,9,global wealth management,global banking and markets,business_segment_review,global wealth management


In [120]:
#drop columns if extracted text is less than 100 letters
finnacial_release_df.drop(finnacial_release_df[finnacial_release_df['extracted_text'].apply(len) <100].index, inplace=True)

In [122]:

print('Shape',finnacial_release_df.shape)
finnacial_release_df.head()

Shape (30, 6)


,start_page,end_page,start_word,end_word,tag,extracted_text
0,1,1,second quarter 2024 earnings release,second quarter 2024 highlights on a reported b...,summary,second quarter 2024 earnings release scotiaban...
1,1,1,second quarter 2024 highlights on a reported b...,second quarter 2024 highlights on an adjusted ...,summary,second quarter 2024 highlights on a reported b...
2,1,1,second quarter 2024 highlights on an adjusted ...,"toronto,",summary,second quarter 2024 highlights on an adjusted ...
3,1,2,"toronto,",financial highlights,summary,"toronto, may 28, 2024 —the bank of nova scotia..."
4,2,2,reported results,business segment review,financial_ighlights,reported results for the three months ended f...




> After checking all the data extracted and noticing some rows had repeated information with no meaning at all, they were removed, and a df was obtained with only 30 rows and 6 features that will be used for training the models.


> Now it's time to load the tables in a format more readable like a table, for that tabula will be used, all the tables will be extracted, cleaned and given the proper format





In [133]:
#check tables format

#first table
tables[0]

#frist table looks in a good shape, some negatives values needs to be
#modifed to look more like real values ex. negatives, nan

,Unnamed: 0,Unnamed: 1,April 30,Unnamed: 2,January 31,Unnamed: 3,April 30.1,Unnamed: 4,April 30.2,Unnamed: 5,April 30.3
0,(Unaudited) ($ millions),NaN,2024(1),NaN,2024(1),NaN,2023(1),NaN,2024(1),NaN,2023(1)
1,Operating results,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Net interest income,$,"4,694",$,"4,773",$,"4,460",$,"9,467",$,"9,023"
3,Non-interest income,NaN,"3,653",NaN,"3,660",NaN,"3,453",NaN,"7,313",NaN,"6,852"
4,Total revenue,$,"8,347",$,"8,433",$,"7,913",$,"16,780",$,"15,875"
5,Provision for credit losses,NaN,"1,007",NaN,962,NaN,709,NaN,"1,969",NaN,"1,347"
6,Non-interest expenses,NaN,"4,711",NaN,"4,739",NaN,"4,574",NaN,"9,450",NaN,"9,035"
7,Income tax expense,NaN,537,NaN,533,NaN,484,NaN,"1,070",NaN,"1,589"
8,Net income,$,"2,092",$,"2,199",$,"2,146",$,"4,291",$,"3,904"
9,Net income attributable to non-controlling int...,NaN,26,NaN,25,NaN,24,NaN,51,NaN,61


In [134]:
#let's check the last table
tables[len(tables)-1]

#this table needs more work done since this table has at least 2 tables merged together
#the library was not able to capture it correctly

,"($ millions) January 31, 2024(1)","April 30, 2023(1)","April 30, 2023(1).1"
0,Constant,Constant,Constant
1,Foreign dollar,Foreign dollar,Foreign dollar
2,(Taxable equivalent basis) Adjusted exchange a...,Adjusted exchange adjusted,Adjusted exchange adjusted
3,"Net interest income $ 2,246 $ 19 $ 2,227","$ 1,999 $ 8 $ 1,991","$ 3,891 $ (82) $ 3,973"
4,Non-interest income 857 6 851,743 (88) 831,"1,535 (163) 1,698"
5,"Total revenue 3,103 25 3,078","2,742 (80) 2,822","5,426 (245) 5,671"
6,Provision for credit losses 574 6 568,436 (3) 439,840 (27) 867
7,"Non-interest expenses 1,563 2 1,561","1,467 (24) 1,491","2,890 (99) 2,989"
8,Income tax expense 192 4 188,174 (10) 184,345 (20) 365
9,Net income $ 774 $ 13 $ 761,$ 665 $ (43) $ 708,"$ 1,351 $ (99) $ 1,450"


In [139]:
tables[2]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,Canadian,International,Wealth,Banking and,NaN,NaN,NaN,NaN
1,($ millions),NaN,Banking(2),Banking(2),Management,Markets,NaN,Other,NaN,Total(2)
2,Reported net income (loss),$,"1,008",$ 695,$ 382 $,428,$,(421),$,"2,092"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),NaN,–,24,2,–,NaN,–,NaN,26
5,Reported net income attributable to equity hol...,NaN,"1,008",671,380,428,NaN,(421),NaN,"2,066"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,NaN,–,–,–,–,NaN,123,NaN,123
8,Reported net income attributable to common sha...,$,"1,008",$ 671,$ 380 $,428,$,(544),$,"1,943"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [404]:

# Function to add dollar sign to numeric values
def add_dollar_sign(value):
    if isinstance(value, str) and value.replace(',', '').isdigit():
        return f'${value}'
    return value


# Define the function to convert values
def add_dollar_negative_sign(value):
    if isinstance(value, str):
        # Check if the value is in parentheses
        match = re.match(r'\(([\d,]+)\)', value)
        if match:
            # Extract the number, convert to negative, and format with dollar sign
            number = match.group(1).replace(',', '')
            return f'${-int(number):,}'
    return value




# Function to create new column names
def add_headers(df):
    new_columns = []
    for col in df.columns:
       # Join the first 3 rows for this column
       if 'Unnamed' in col:
          joined_name = ' '.join(df[col].iloc[:2].dropna().astype(str))
       else:
          joined_name =  col + ' ' + ' '.join(df[col].iloc[:2].dropna().astype(str))

       new_columns.append(joined_name)

    return new_columns


# Function to clean column names from (number) and .number
def remove_number_header(name):
    # Remove patterns like (number) and .number
    cleaned_name = re.sub(r'\(\d+\)', '', name)  # Remove (number)
    cleaned_name = re.sub(r'\.\d+', '', cleaned_name)  # Remove .number
    return cleaned_name.strip()



#if any row has double text then join them
def join_row_double_text(df):
    rows = []

    for i in range(len(df)):
        if df.iloc[i, 0] == 'Adjustments:':
          continue


        if i > 0 and df.iloc[i-1][1:].isna().all():

            # Previous row is all NaN, current row contains values
            merged_name = df.iloc[i-1, 0] + ' ' + df.iloc[i, 0] if not pd.isna(df.iloc[i, 0]) else df.iloc[i-1, 0]
            # Create a new row with merged column name and current row's values
            new_row = [merged_name] + list(df.iloc[i, 1:])
            rows.append(new_row)
        else:
            # Normal row, just add it
            rows.append(df.iloc[i].tolist())

    # Create a new DataFrame from the rows
    result_df = pd.DataFrame(rows, columns=df.columns)

    return result_df

# Function to split DataFrame based on specific text
def split_dataframe_on_text(df, text, check_columns):
    split_indices = []
    tables = []

    # Track the start of the current table
    start_idx = 0

    for i in range(len(df)):
        row = df.iloc[i]
        # Check if column 0 is NaN or null
        if pd.isna(row[0]):
            # Check other specified columns for the text
            found = any(text in str(row[col]) for col in check_columns)
        else:
            # Check if the text is in column 0
            found = text in str(row[0])

        if found:
            # Check if this row contains the specified text
            if i > start_idx:
                split_indices.append(i)
                tables.append(df.iloc[start_idx:i])
                start_idx = i

    # Append the last segment
    if start_idx < len(df):
        tables.append(df.iloc[start_idx:])

    return tables






In [405]:
new_tables=[]

In [415]:

#clean the tables
df = tables[0]

# Apply the function to all cells in the DataFrame to add dollar sign
df = df.applymap(add_dollar_sign)
#add $- to negative values
df = df.applymap(add_dollar_negative_sign)
# remove column names from (number) and .number
df.columns = [remove_number_header(col) for col in df.columns]



<ipython-input-415-4fe49e00c0ab>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(add_dollar_sign)
<ipython-input-415-4fe49e00c0ab>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(add_dollar_negative_sign)


In [417]:
def remove_dollar_sign(df):
    # Define a function to process each cell
    def remove_dollar_sign_from_cell(cell):
        if isinstance(cell, str) and cell.endswith('$'):
            # Remove the dollar sign from the end of the string
            return cell.rstrip('$')
        return cell

    # Apply the function to each cell in the DataFrame
    return df.applymap(remove_dollar_sign_from_cell)


In [418]:
remove_dollar_sign(tables[0])

<ipython-input-417-91d1f838e962>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(remove_dollar_sign_from_cell)


,($ millions),Canadian Banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
0,Reported net income (loss),"$1,008",$ 695,$ 382,$428,$-421,"$2,092"
2,Net income attributable to non-controlling interests in subsidiaries (NCI),–,$24,$2,–,–,$26
3,Reported net income attributable to equity holders,"$1,008",$671,$380,$428,$-421,"$2,066"
5,Reported net income attributable to preferred shareholders and other equity instrument holders,–,–,–,–,$123,$123
6,Reported net income attributable to common shareholders,"$1,008",$ 671,$ 380,$428,$-544,"$1,943"
8,Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets,$1,$8,$9,–,–,$18
9,Total non-interest expenses adjustments (Pre-tax),$1,$8,$9,–,–,$18
10,Total impact of adjusting items on net income before taxes,$1,$8,$9,–,–,$18
11,Impact of adjusting items on income tax expense,$-1,$-2,$-2,–,–,$-5
12,Total impact of adjusting items on net income,–,$6,$7,–,–,$13


In [414]:
tables[0]

,($ millions),Canadian Banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
0,Reported net income (loss),"$1,008",$ 695,$ 382 $,$428,$-421,"$2,092"
2,Net income attributable to non-controlling interests in subsidiaries (NCI),–,$24,$2,–,–,$26
3,Reported net income attributable to equity holders,"$1,008",$671,$380,$428,$-421,"$2,066"
5,Reported net income attributable to preferred shareholders and other equity instrument holders,–,–,–,–,$123,$123
6,Reported net income attributable to common shareholders,"$1,008",$ 671,$ 380 $,$428,$-544,"$1,943"
8,Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets,$1,$8,$9,–,–,$18
9,Total non-interest expenses adjustments (Pre-tax),$1,$8,$9,–,–,$18
10,Total impact of adjusting items on net income before taxes,$1,$8,$9,–,–,$18
11,Impact of adjusting items on income tax expense,$-1,$-2,$-2,–,–,$-5
12,Total impact of adjusting items on net income,–,$6,$7,–,–,$13


In [416]:
df

,($ millions),Canadian Banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
0,Reported net income (loss),"$1,008",$ 695,$ 382 $,$428,$-421,"$2,092"
2,Net income attributable to non-controlling interests in subsidiaries (NCI),–,$24,$2,–,–,$26
3,Reported net income attributable to equity holders,"$1,008",$671,$380,$428,$-421,"$2,066"
5,Reported net income attributable to preferred shareholders and other equity instrument holders,–,–,–,–,$123,$123
6,Reported net income attributable to common shareholders,"$1,008",$ 671,$ 380 $,$428,$-544,"$1,943"
8,Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets,$1,$8,$9,–,–,$18
9,Total non-interest expenses adjustments (Pre-tax),$1,$8,$9,–,–,$18
10,Total impact of adjusting items on net income before taxes,$1,$8,$9,–,–,$18
11,Impact of adjusting items on income tax expense,$-1,$-2,$-2,–,–,$-5
12,Total impact of adjusting items on net income,–,$6,$7,–,–,$13


In [408]:

# Apply the function to all cells in the DataFrame
df = df.applymap(add_dollar_sign)
#add $- to negative values
df = df.applymap(add_dollar_negative_sign)
# Apply the function to get new column names
df.columns = add_headers(df)
#remove row 0 and 1 the headers were added so is not necessary
df.drop(index=[0, 1], inplace=True)
# remove column names from (number) and .number
df.columns = [remove_number_header(col) for col in df.columns]
#remove second column , it does not contain any valuable info
df.drop(df.columns[1], axis=1, inplace=True)
# Apply the function to merge columns
df = join_row_double_text(df)
# Drop rows where all columns after the first one are NaN
df.dropna(subset=df.columns[1:], how='all', inplace=True)
# Apply the function to split the DataFrame
# Apply the function to split the DataFrame
split_text = "For the three months"
check_columns = [0, 1, 2, 3, 4, 5, 6]  # List of column indices to check
#add resutls to new table
new_tables.append(split_dataframe_on_text(df, split_text, check_columns))




<ipython-input-408-453ba8780bd3>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(add_dollar_sign)
<ipython-input-408-453ba8780bd3>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(add_dollar_negative_sign)


KeyError: '[0, 1] not found in axis'

In [373]:
df

,($ millions),Canadian Banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
0,Reported net income (loss),"$1,008",$ 695,$ 382 $,$428,$-421,"$2,092"
2,Net income attributable to non-controlling interests in subsidiaries (NCI),–,$24,$2,–,–,$26
3,Reported net income attributable to equity holders,"$1,008",$671,$380,$428,$-421,"$2,066"
5,Reported net income attributable to preferred shareholders and other equity instrument holders,–,–,–,–,$123,$123
6,Reported net income attributable to common shareholders,"$1,008",$ 671,$ 380 $,$428,$-544,"$1,943"
8,Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets,$1,$8,$9,–,–,$18
9,Total non-interest expenses adjustments (Pre-tax),$1,$8,$9,–,–,$18
10,Total impact of adjusting items on net income before taxes,$1,$8,$9,–,–,$18
11,Impact of adjusting items on income tax expense,$-1,$-2,$-2,–,–,$-5
12,Total impact of adjusting items on net income,–,$6,$7,–,–,$13


<ipython-input-397-69794f226692>:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.isna(row[0]):
<ipython-input-397-69794f226692>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  found = text in str(row[0])
<ipython-input-397-69794f226692>:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  found = any(text in str(row[col]) for col in check_columns)


In [402]:
tables[1]

,($ millions),Canadian Banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
23,NaN,NaN,"For the three months ended January 31, 2024(1)",NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,Global,Global,NaN,NaN
25,NaN,Canadian,International,Wealth,Banking and,NaN,NaN
26,($ millions),Banking(2),Banking(2),Management,Markets,Other,Total(2)
27,Reported net income (loss),"$1,095",$ 768,$ 371 $,$439,$-474,"$2,199"
29,Net income attributable to non-controlling interests in subsidiaries (NCI),–,$22,$3,–,–,$25
30,Reported net income attributable to equity holders,"$1,095",$746,$368,$439,$-474,"$2,174"
32,Reported net income attributable to preferred shareholders and other equity instrument holders,$1,$1,–,$1,$105,$108
33,Reported net income attributable to common shareholders,"$1,094",$ 745,$ 368 $,$438,$-579,"$2,066"
35,Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets,$1,$8,$9,–,–,$18


In [368]:

# Apply the function to split the DataFrame
split_dfs = split_dataframe(df)

In [366]:
#dataframe with 1 row or less convert to string

for
split_dfs[0]


split_dfs[3]

,($ millions),Canadian Banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
25,NaN,Canadian,International,Wealth,Banking and,NaN,NaN
26,($ millions),Banking(2),Banking(2),Management,Markets,Other,Total(2)
27,Reported net income (loss),"$1,095",$ 768,$ 371 $,$439,$-474,"$2,199"
29,Net income attributable to non-controlling interests in subsidiaries (NCI),–,$22,$3,–,–,$25
30,Reported net income attributable to equity holders,"$1,095",$746,$368,$439,$-474,"$2,174"
32,Reported net income attributable to preferred shareholders and other equity instrument holders,$1,$1,–,$1,$105,$108
33,Reported net income attributable to common shareholders,"$1,094",$ 745,$ 368 $,$438,$-579,"$2,066"
35,Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets,$1,$8,$9,–,–,$18
36,Total non-interest expenses adjustments (Pre-tax),$1,$8,$9,–,–,$18
37,Total impact of adjusting items on net income before taxes,$1,$8,$9,–,–,$18


In [ ]:
drop any row that has nan after the 1 column

In [ ]:
#len of the document
print('Total pages', len(file_text))


#words per page
for index , page in enumerate(file_text):
    words_per_page = len(page.split())
    print(f'Total words page #{index} :{ words_per_page}')

#amount of words in the document
total_words = sum(len(page.split()) for page in file_text)
print('\nTotal Words:', total_words)


In [ ]:
def format_currency(value):
    if pd.isna(value):
        return
    return locale.currency(value, grouping=True).split('.')[0]
# Function to add dollar sign before numbers
def add_dollar_sign(text):
    # Function to add dollar sign to numbers
    def replace_with_dollar(match):
        return f'${match.group(0)}'

    # Use regular expression to find numbers
    return re.sub(r'\b\d+(?:,\d{3})*(?:\.\d+)?\b', replace_with_dollar, text)

def split_dollars(row):
    if '$' in row:
        parts = row.split('$')
        # Remove empty strings and strip whitespaces
        return [parts[0].strip()] + [part.strip() for part in parts[1:]]
    else:
        return [row] + [''] * 3

In [ ]:
# Replace 'your_pdf_file.pdf' with the path to your PDF file
pdf_path = "Q224.pdf"

# Extract all tables from the PDF
tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)


In [ ]:
type(tables)

list

In [ ]:
len(tables)

6

In [ ]:
tables[0] #Financial Highlights #Reported Results

,Unnamed: 0,Unnamed: 1,April 30,Unnamed: 2,January 31,Unnamed: 3,April 30.1,Unnamed: 4,April 30.2,Unnamed: 5,April 30.3
0,(Unaudited) ($ millions),NaN,2024(1),NaN,2024(1),NaN,2023(1),NaN,2024(1),NaN,2023(1)
1,Operating results,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Net interest income,$,"4,694",$,"4,773",$,"4,460",$,"9,467",$,"9,023"
3,Non-interest income,NaN,"3,653",NaN,"3,660",NaN,"3,453",NaN,"7,313",NaN,"6,852"
4,Total revenue,$,"8,347",$,"8,433",$,"7,913",$,"16,780",$,"15,875"
5,Provision for credit losses,NaN,"1,007",NaN,962,NaN,709,NaN,"1,969",NaN,"1,347"
6,Non-interest expenses,NaN,"4,711",NaN,"4,739",NaN,"4,574",NaN,"9,450",NaN,"9,035"
7,Income tax expense,NaN,537,NaN,533,NaN,484,NaN,"1,070",NaN,"1,589"
8,Net income,$,"2,092",$,"2,199",$,"2,146",$,"4,291",$,"3,904"
9,Net income attributable to non-controlling int...,NaN,26,NaN,25,NaN,24,NaN,51,NaN,61


In [ ]:
table1=tables[1]

In [ ]:
# tables[0]
for colIndex, colName in enumerate(table1):

  for rowIndex, text in enumerate(table1[colName]):
    if pd.isna(text):
      continue

    #remove number comma to be able to identify as number
    text = text.replace(',','')

    if len(text) > 2:
      if text[1].isnumeric():
        # print(text,len(text))
        text= text.replace('(', ' -').replace(')', '')

        indexHypen = text.find('-')
        if  indexHypen > -1 :
          text= text[:indexHypen+1] + '$' + text[indexHypen +1:]
    if text.isnumeric():
      text = '$' + text

    table1.loc[rowIndex, colName] = text
    print('change',table1.loc[rowIndex, colName], text )  # Update the DataFrame




change ($ millions) ($ millions)
change Reported Results Reported Results
change Net interest income Net interest income
change Non-interest income Non-interest income
change Total revenue Total revenue
change Provision for credit losses Provision for credit losses
change Non-interest expenses Non-interest expenses
change Income before taxes Income before taxes
change Income tax expense Income tax expense
change Net income Net income
change Net income attributable to non-controlling interests in subsidiaries (NCI) Net income attributable to non-controlling interests in subsidiaries (NCI)
change Net income attributable to equity holders Net income attributable to equity holders
change Net income attributable to preferred shareholders and other equity Net income attributable to preferred shareholders and other equity
change instrument holders instrument holders
change Net income attributable to common shareholders Net income attributable to common shareholders
change Diluted earnings per

In [ ]:
table1

,Unnamed: 0,Unnamed: 1,April 30,Unnamed: 2,January 31,Unnamed: 3,April 30.1,Unnamed: 4,April 30.2,Unnamed: 5,April 30.3
0,($ millions),NaN,$2024 -$$$$1,NaN,$2024 -$$$$1,NaN,$2023 -$$$$1,NaN,$2024 -$$$$1,NaN,$2023 -$$$$1
1,Reported Results,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Net interest income,$,$4694,$,$4773,$,$4460,$,$9467,$,$9023
3,Non-interest income,NaN,$3653,NaN,$3660,NaN,$3453,NaN,$7313,NaN,$6852
4,Total revenue,NaN,$8347,NaN,$8433,NaN,$7913,NaN,$16780,NaN,$15875
5,Provision for credit losses,NaN,$1007,NaN,$962,NaN,$709,NaN,$1969,NaN,$1347
6,Non-interest expenses,NaN,$4711,NaN,$4739,NaN,$4574,NaN,$9450,NaN,$9035
7,Income before taxes,NaN,$2629,NaN,$2732,NaN,$2630,NaN,$5361,NaN,$5493
8,Income tax expense,NaN,$537,NaN,$533,NaN,$484,NaN,$1070,NaN,$1589
9,Net income,$,$2092,$,$2199,$,$2146,$,$4291,$,$3904


In [ ]:
tables[1] #Reconciliation of reported and adjusted results and diluted earnings per share

,Unnamed: 0,Unnamed: 1,April 30,Unnamed: 2,January 31,Unnamed: 3,April 30.1,Unnamed: 4,April 30.2,Unnamed: 5,April 30.3
0,($ millions),NaN,2024(1),NaN,2024(1),NaN,2023(1),NaN,2024(1),NaN,2023(1)
1,Reported Results,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Net interest income,$,"4,694",$,"4,773",$,"4,460",$,"9,467",$,"9,023"
3,Non-interest income,NaN,"3,653",NaN,"3,660",NaN,"3,453",NaN,"7,313",NaN,"6,852"
4,Total revenue,NaN,"8,347",NaN,"8,433",NaN,"7,913",NaN,"16,780",NaN,"15,875"
5,Provision for credit losses,NaN,"1,007",NaN,962,NaN,709,NaN,"1,969",NaN,"1,347"
6,Non-interest expenses,NaN,"4,711",NaN,"4,739",NaN,"4,574",NaN,"9,450",NaN,"9,035"
7,Income before taxes,NaN,"2,629",NaN,"2,732",NaN,"2,630",NaN,"5,361",NaN,"5,493"
8,Income tax expense,NaN,537,NaN,533,NaN,484,NaN,"1,070",NaN,"1,589"
9,Net income,$,"2,092",$,"2,199",$,"2,146",$,"4,291",$,"3,904"


In [ ]:
tables[2] #Reconciliation of reported and adjusted results by business line

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,Canadian,International,Wealth,Banking and,NaN,NaN,NaN,NaN
1,($ millions),NaN,Banking(2),Banking(2),Management,Markets,NaN,Other,NaN,Total(2)
2,Reported net income (loss),$,"1,008",$ 695,$ 382 $,428,$,(421),$,"2,092"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),NaN,–,24,2,–,NaN,–,NaN,26
5,Reported net income attributable to equity hol...,NaN,"1,008",671,380,428,NaN,(421),NaN,"2,066"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,NaN,–,–,–,–,NaN,123,NaN,123
8,Reported net income attributable to common sha...,$,"1,008",$ 671,$ 380 $,428,$,(544),$,"1,943"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tables[3] #page 9

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Canadian,NaN,International,NaN,Wealth,Banking and,NaN,NaN,NaN,NaN
1,($ millions),NaN,Banking(2),NaN,Banking(2),NaN,Management,Markets,NaN,Other,NaN,Total(2)
2,Reported net income (loss),$,"1,055",$,657,$,356 $,401,$,(323),$,"2,146"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),NaN,–,NaN,21,NaN,3,–,NaN,–,NaN,24
5,Reported net income attributable to equity hol...,NaN,"1,055",NaN,636,NaN,353,401,NaN,(323),NaN,"2,122"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,NaN,1,NaN,1,NaN,1,1,NaN,100,NaN,104
8,Reported net income attributable to common sha...,$,"1,054",$,635,$,352 $,400,$,(423),$,"2,018"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tables[4] #page 10

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Canadian,NaN,International,NaN,Wealth,Banking and,NaN,NaN,NaN,NaN
1,($ millions),NaN,Banking(2),NaN,Banking(2),NaN,Management,Markets,NaN,Other,NaN,Total(2)
2,Reported net income (loss),$,"2,141",$,"1,336",$,743 $,920,$,"(1,236)",$,"3,904"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),NaN,–,NaN,56,NaN,5,–,NaN,–,NaN,61
5,Reported net income attributable to equity hol...,NaN,"2,141",NaN,"1,280",NaN,738,920,NaN,"(1,236)",NaN,"3,843"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,NaN,2,NaN,2,NaN,1,2,NaN,198,NaN,205
8,Reported net income attributable to common sha...,$,"2,139",$,"1,278",$,737 $,918,$,"(1,434)",$,"3,638"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tables[5]

,"($ millions) January 31, 2024(1)","April 30, 2023(1)","April 30, 2023(1).1"
0,Constant,Constant,Constant
1,Foreign dollar,Foreign dollar,Foreign dollar
2,(Taxable equivalent basis) Adjusted exchange a...,Adjusted exchange adjusted,Adjusted exchange adjusted
3,"Net interest income $ 2,246 $ 19 $ 2,227","$ 1,999 $ 8 $ 1,991","$ 3,891 $ (82) $ 3,973"
4,Non-interest income 857 6 851,743 (88) 831,"1,535 (163) 1,698"
5,"Total revenue 3,103 25 3,078","2,742 (80) 2,822","5,426 (245) 5,671"
6,Provision for credit losses 574 6 568,436 (3) 439,840 (27) 867
7,"Non-interest expenses 1,563 2 1,561","1,467 (24) 1,491","2,890 (99) 2,989"
8,Income tax expense 192 4 188,174 (10) 184,345 (20) 365
9,Net income $ 774 $ 13 $ 761,$ 665 $ (43) $ 708,"$ 1,351 $ (99) $ 1,450"


In [ ]:

# Iterate over extracted tables and save them to CSV files
for i, table in enumerate(tables):
    print(f"Table {i+1}")
    # Save each table to a separate CSV file
    table.to_csv(f"table_{i+1}.csv", index=False)

Table 1
Table 2
Table 3
Table 4
Table 5
Table 6


### 5. Exploratory Data Analysis

### 6. Train and Split

NameError: name 'java' is not defined

### 7. Modeling

### 8. Metrics

### 9. Testing

### 10. Conclusion